In [ ]:
# Paso 1: Subir archivo .sql
from google.colab import files
uploaded = files.upload()

Saving RESPALDO_Marcaciones_20250707.sql to RESPALDO_Marcaciones_20250707 (5).sql


In [ ]:
# Paso 2: Mostrar archivos en entorno para confirmar el nombre
import os
print("Archivos en el entorno:", os.listdir())

Archivos en el entorno: ['.config', 'RESPALDO_Marcaciones_20250707.sql', '.ipynb_checkpoints', 'sample_data']


In [ ]:
import pandas as pd
import re
from google.colab import files

# Archivo SQL específico
archivo_sql = 'RESPALDO_Marcaciones_20250707.sql'

# Leer contenido del archivo
with open(archivo_sql, 'r', encoding='utf-8') as f:
    contenido = f.read()

# Buscar columnas del primer INSERT INTO iclock_transaction
patron_columnas = re.compile(
    r'insert\s+into\s+[`"]?iclock_transaction[`"]?\s*\((.*?)\)\s*values',
    flags=re.DOTALL | re.IGNORECASE
)
columnas_match = patron_columnas.search(contenido)
if not columnas_match:
    raise ValueError("No se encontró patrón 'INSERT INTO iclock_transaction(...) VALUES'.")

columnas = [col.strip('`" ') for col in columnas_match.group(1).split(',')]
print(f"Columnas encontradas ({len(columnas)}): {columnas}")

# Extraer todos los valores de todos los INSERT INTO iclock_transaction
patron_valores = re.compile(
    r'insert\s+into\s+[`"]?iclock_transaction[`"]?\s*\(.*?\)\s*values\s*(.*?);',
    flags=re.DOTALL | re.IGNORECASE
)
valores_match = patron_valores.findall(contenido)
if not valores_match:
    raise ValueError("No se encontraron valores en los INSERT INTO iclock_transaction.")

valores_brutos = ','.join(valores_match)

# Extraer filas (grupos entre paréntesis)
filas_raw = re.findall(r'\((.*?)\)', valores_brutos, flags=re.DOTALL)
print(f"Filas extraídas: {len(filas_raw)}")

# Procesar cada fila para convertir valores
filas = []
for fila in filas_raw:
    fila = fila.replace("'null'", "None").replace("NULL", "None")
    partes = re.findall(r"(?:'[^']*'|[^,]+)", fila)
    partes = [eval(p.strip()) if p.strip() != 'None' else None for p in partes]
    filas.append(partes)

# Crear DataFrame
df = pd.DataFrame(filas, columns=columnas)

# Convertir columna punch_time a datetime
df['punch_time'] = pd.to_datetime(df['punch_time'], errors='coerce')

# Definir filtro emp_code y rango de fechas
emp_code_filtrar = '0913467791'
fecha_inicio = pd.Timestamp('2025-04-01 00:00:00')
fecha_fin = pd.Timestamp('2025-07-07 23:59:59')

# Aplicar filtro
df_filtrado = df[
    (df['emp_code'] == emp_code_filtrar) &
    (df['punch_time'] >= fecha_inicio) &
    (df['punch_time'] <= fecha_fin)
]

# Guardar Excel con datos filtrados
archivo_excel_filtrado = f'iclock_transaction_filtrado_{emp_code_filtrar}.xlsx'
df_filtrado.to_excel(archivo_excel_filtrado, index=False)

print(f"✅ Excel filtrado generado: {archivo_excel_filtrado}")

# Descargar archivo filtrado
files.download(archivo_excel_filtrado)


Columnas encontradas (26): ['id', 'emp_code', 'punch_time', 'punch_state', 'verify_type', 'work_code', 'terminal_sn', 'terminal_alias', 'area_alias', 'longitude', 'latitude', 'gps_location', 'mobile', 'source', 'purpose', 'crc', 'is_attendance', 'reserved', 'upload_time', 'sync_status', 'sync_time', 'is_mask', 'temperature', 'emp_id', 'terminal_id', 'company_code']
Filas extraídas: 63260
✅ Excel filtrado generado: iclock_transaction_filtrado_0913467791.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>